# 🤖 Chatbot Avancé avec LangChain et HuggingFace

## 📋 **Objectifs**
- Intégrer **LangChain** pour la gestion des conversations
- Utiliser **HuggingFace Transformers** pour les modèles de langage
- Créer un chatbot intelligent pour recommandations Airbnb
- Implémenter la mémoire conversationnelle
- Ajouter des capacités de RAG (Retrieval-Augmented Generation)

## 🛠️ **Technologies Utilisées**
- **LangChain** : Framework pour applications LLM
- **HuggingFace Transformers** : Modèles de langage pré-entraînés
- **FAISS** : Recherche vectorielle rapide
- **Sentence Transformers** : Embeddings sémantiques

## 📦 **Installation des Dépendances**

In [ ]:
# Installation des packages nécessaires
!pip install langchain langchain-community langchain-huggingface
!pip install transformers torch sentence-transformers
!pip install faiss-cpu chromadb
!pip install gradio streamlit

## 📚 **Imports et Configuration**

In [ ]:
import os
import pandas as pd
import numpy as np
from typing import List, Dict, Any
import warnings
warnings.filterwarnings('ignore')

# LangChain imports
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate

# HuggingFace imports
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    pipeline, BitsAndBytesConfig
)
import torch

# Interface utilisateur
import gradio as gr

print("✅ Tous les imports réussis !")

## 🔧 **Configuration du Modèle HuggingFace**

In [ ]:
class AirbnbChatbotConfig:
    """Configuration pour le chatbot Airbnb"""
    
    # Modèles HuggingFace recommandés
    MODELS = {
        'small': 'microsoft/DialoGPT-small',  # Rapide, léger
        'medium': 'microsoft/DialoGPT-medium',  # Équilibré
        'french': 'dbmdz/bert-base-french-europeana-cased',  # Français
        'multilingual': 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
    }
    
    # Configuration par défaut
    DEFAULT_MODEL = 'small'
    MAX_LENGTH = 512
    TEMPERATURE = 0.7
    TOP_P = 0.9
    
    # Embeddings pour la recherche sémantique
    EMBEDDING_MODEL = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
    
    # Prompts en français
    SYSTEM_PROMPT = """
    Tu es un assistant intelligent spécialisé dans les recommandations d'hébergements Airbnb 
    pour la Tunisie, particulièrement Hammamet et Jerba.
    
    Tes responsabilités :
    - Recommander des hébergements basés sur les préférences utilisateur
    - Analyser les avis clients pour donner des conseils
    - Fournir des informations sur les destinations
    - Être amical, professionnel et informatif
    
    Réponds toujours en français et sois concis mais complet.
    """

config = AirbnbChatbotConfig()
print("⚙️ Configuration initialisée")

## 🧠 **Initialisation du Modèle HuggingFace**

In [ ]:
def initialize_huggingface_model(model_size='small'):
    """
    Initialise le modèle HuggingFace pour la génération de texte
    
    Args:
        model_size (str): Taille du modèle ('small', 'medium', 'french')
    
    Returns:
        HuggingFacePipeline: Pipeline LangChain configuré
    """
    print(f"🔄 Chargement du modèle {model_size}...")
    
    model_name = config.MODELS[model_size]
    
    try:
        # Configuration pour optimiser la mémoire
        device = 0 if torch.cuda.is_available() else -1
        
        # Créer le pipeline de génération de texte
        text_generation_pipeline = pipeline(
            "text-generation",
            model=model_name,
            tokenizer=model_name,
            max_length=config.MAX_LENGTH,
            temperature=config.TEMPERATURE,
            top_p=config.TOP_P,
            device=device,
            do_sample=True,
            pad_token_id=50256  # Pour éviter les warnings
        )
        
        # Wrapper LangChain
        llm = HuggingFacePipeline(
            pipeline=text_generation_pipeline,
            model_kwargs={
                "temperature": config.TEMPERATURE,
                "max_length": config.MAX_LENGTH
            }
        )
        
        print(f"✅ Modèle {model_name} chargé avec succès !")
        return llm
        
    except Exception as e:
        print(f"❌ Erreur lors du chargement du modèle : {e}")
        print("🔄 Tentative avec un modèle plus simple...")
        
        # Fallback vers un modèle plus simple
        simple_pipeline = pipeline(
            "text-generation",
            model="gpt2",
            max_length=256,
            device=device
        )
        
        return HuggingFacePipeline(pipeline=simple_pipeline)

# Initialiser le modèle
llm = initialize_huggingface_model('small')

## 🔍 **Configuration des Embeddings et Base de Connaissances**

In [ ]:
def create_knowledge_base():
    """
    Crée une base de connaissances vectorielle pour le RAG
    
    Returns:
        FAISS: Base de données vectorielle
    """
    print("📚 Création de la base de connaissances...")
    
    # Données d'exemple sur les hébergements Airbnb Tunisie
    knowledge_data = [
        {
            "content": "Hammamet est une destination balnéaire populaire en Tunisie, connue pour ses plages de sable fin et sa médina historique. Les hébergements Airbnb y offrent souvent des vues sur mer et un accès facile aux attractions touristiques.",
            "location": "Hammamet",
            "type": "destination_info"
        },
        {
            "content": "Jerba est une île tunisienne réputée pour son climat doux, ses plages magnifiques et son patrimoine culturel riche. Les hébergements traditionnels avec architecture locale sont très appréciés des visiteurs.",
            "location": "Jerba", 
            "type": "destination_info"
        },
        {
            "content": "Pour un séjour familial, privilégiez les hébergements avec piscine, cuisine équipée et proximité des plages. Les villas avec jardin sont idéales pour les groupes.",
            "location": "Général",
            "type": "recommendation"
        },
        {
            "content": "Les couples apprécient les riads traditionnels avec terrasse privée, les appartements avec vue sur mer et les hébergements dans les médinas pour une expérience authentique.",
            "location": "Général",
            "type": "recommendation"
        },
        {
            "content": "La meilleure période pour visiter la Tunisie est d'avril à juin et de septembre à novembre. Les prix des hébergements sont plus avantageux hors saison estivale.",
            "location": "Général",
            "type": "travel_tips"
        }
    ]
    
    # Créer un DataFrame
    df = pd.DataFrame(knowledge_data)
    
    # Charger avec LangChain
    loader = DataFrameLoader(df, page_content_column="content")
    documents = loader.load()
    
    # Diviser les documents
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=50
    )
    texts = text_splitter.split_documents(documents)
    
    # Créer les embeddings
    embeddings = HuggingFaceEmbeddings(
        model_name=config.EMBEDDING_MODEL,
        model_kwargs={'device': 'cpu'}  # Utiliser CPU pour la compatibilité
    )
    
    # Créer la base vectorielle FAISS
    vectorstore = FAISS.from_documents(texts, embeddings)
    
    print(f"✅ Base de connaissances créée avec {len(texts)} documents")
    return vectorstore

# Créer la base de connaissances
vectorstore = create_knowledge_base()

## 🧠 **Configuration de la Mémoire Conversationnelle**

In [ ]:
def create_conversation_chain(llm, vectorstore):
    """
    Crée une chaîne conversationnelle avec mémoire et RAG
    
    Args:
        llm: Modèle de langage
        vectorstore: Base de données vectorielle
    
    Returns:
        ConversationalRetrievalChain: Chaîne conversationnelle
    """
    print("🔗 Configuration de la chaîne conversationnelle...")
    
    # Mémoire conversationnelle
    memory = ConversationBufferMemory(
        memory_key="chat_history",
        return_messages=True,
        output_key="answer"
    )
    
    # Créer la chaîne conversationnelle
    qa_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vectorstore.as_retriever(
            search_kwargs={"k": 3}  # Récupérer les 3 documents les plus pertinents
        ),
        memory=memory,
        return_source_documents=True,
        verbose=True
    )
    
    print("✅ Chaîne conversationnelle configurée")
    return qa_chain

# Créer la chaîne conversationnelle
conversation_chain = create_conversation_chain(llm, vectorstore)

## 🤖 **Classe Chatbot Principal**

In [ ]:
class AirbnbLangChainChatbot:
    """
    Chatbot Airbnb utilisant LangChain et HuggingFace
    """
    
    def __init__(self, conversation_chain):
        self.conversation_chain = conversation_chain
        self.conversation_history = []
        
    def chat(self, user_input: str) -> Dict[str, Any]:
        """
        Traite une question utilisateur et retourne une réponse
        
        Args:
            user_input (str): Question de l'utilisateur
        
        Returns:
            Dict: Réponse avec métadonnées
        """
        try:
            # Obtenir la réponse de la chaîne conversationnelle
            result = self.conversation_chain({
                "question": user_input
            })
            
            # Extraire la réponse
            answer = result.get("answer", "Désolé, je n'ai pas pu traiter votre demande.")
            source_docs = result.get("source_documents", [])
            
            # Ajouter à l'historique
            self.conversation_history.append({
                "user": user_input,
                "bot": answer,
                "sources": len(source_docs)
            })
            
            return {
                "answer": answer,
                "sources": source_docs,
                "confidence": self._calculate_confidence(answer, source_docs)
            }
            
        except Exception as e:
            error_msg = f"Erreur lors du traitement : {str(e)}"
            print(f"❌ {error_msg}")
            
            return {
                "answer": "Désolé, j'ai rencontré un problème technique. Pouvez-vous reformuler votre question ?",
                "sources": [],
                "confidence": 0.0
            }
    
    def _calculate_confidence(self, answer: str, sources: List) -> float:
        """
        Calcule un score de confiance basé sur la réponse et les sources
        """
        if not answer or "désolé" in answer.lower():
            return 0.2
        
        # Plus il y a de sources, plus la confiance est élevée
        source_score = min(len(sources) * 0.3, 0.9)
        
        # Longueur de la réponse (réponses plus détaillées = plus de confiance)
        length_score = min(len(answer) / 200, 0.5)
        
        return min(source_score + length_score + 0.3, 1.0)
    
    def get_conversation_history(self) -> List[Dict]:
        """Retourne l'historique de conversation"""
        return self.conversation_history
    
    def clear_history(self):
        """Efface l'historique de conversation"""
        self.conversation_history = []
        # Réinitialiser la mémoire de la chaîne
        self.conversation_chain.memory.clear()

# Initialiser le chatbot
chatbot = AirbnbLangChainChatbot(conversation_chain)
print("🤖 Chatbot LangChain initialisé avec succès !")

## 🧪 **Tests du Chatbot**

In [ ]:
def test_chatbot():
    """
    Teste le chatbot avec différentes questions
    """
    print("🧪 Tests du chatbot LangChain...")
    print("=" * 50)
    
    test_questions = [
        "Bonjour ! Peux-tu me recommander un hébergement à Hammamet ?",
        "Quels sont les avantages de Jerba pour des vacances ?",
        "Je voyage avec ma famille, que me conseilles-tu ?",
        "Quelle est la meilleure période pour visiter la Tunisie ?",
        "Merci pour tes conseils !"
    ]
    
    for i, question in enumerate(test_questions, 1):
        print(f"\n🙋 Question {i}: {question}")
        
        response = chatbot.chat(question)
        
        print(f"🤖 Réponse: {response['answer']}")
        print(f"📊 Confiance: {response['confidence']:.2f}")
        print(f"📚 Sources utilisées: {len(response['sources'])}")
        print("-" * 30)
    
    # Afficher l'historique
    print("\n📜 Historique de conversation:")
    for i, conv in enumerate(chatbot.get_conversation_history(), 1):
        print(f"{i}. User: {conv['user'][:50]}...")
        print(f"   Bot: {conv['bot'][:50]}...")

# Lancer les tests
test_chatbot()

## 🎨 **Interface Gradio Interactive**

In [ ]:
def create_gradio_interface():
    """
    Crée une interface Gradio pour le chatbot
    """
    def chat_interface(message, history):
        """
        Interface de chat pour Gradio
        """
        if not message.strip():
            return history, ""
        
        # Obtenir la réponse du chatbot
        response = chatbot.chat(message)
        
        # Ajouter à l'historique Gradio
        history.append([message, response['answer']])
        
        return history, ""
    
    def clear_chat():
        """
        Efface l'historique de chat
        """
        chatbot.clear_history()
        return [], ""
    
    # Créer l'interface Gradio
    with gr.Blocks(
        title="🏠 Chatbot Airbnb Tunisie - LangChain + HuggingFace",
        theme=gr.themes.Soft()
    ) as interface:
        
        gr.Markdown("""
        # 🤖 Assistant Airbnb Intelligent
        
        Powered by **LangChain** + **HuggingFace** 🚀
        
        Posez-moi vos questions sur les hébergements Airbnb en Tunisie !
        """)
        
        chatbot_interface = gr.Chatbot(
            label="💬 Conversation",
            height=400,
            show_label=True
        )
        
        with gr.Row():
            msg = gr.Textbox(
                label="Votre message",
                placeholder="Tapez votre question ici...",
                scale=4
            )
            send_btn = gr.Button("Envoyer 📤", scale=1)
        
        with gr.Row():
            clear_btn = gr.Button("Effacer 🗑️")
            
        # Exemples de questions
        gr.Examples(
            examples=[
                "Recommande-moi un hébergement à Hammamet",
                "Quels sont les avantages de Jerba ?",
                "Je voyage en famille, que conseilles-tu ?",
                "Quelle est la meilleure période pour visiter ?"
            ],
            inputs=msg
        )
        
        # Événements
        send_btn.click(
            chat_interface,
            inputs=[msg, chatbot_interface],
            outputs=[chatbot_interface, msg]
        )
        
        msg.submit(
            chat_interface,
            inputs=[msg, chatbot_interface],
            outputs=[chatbot_interface, msg]
        )
        
        clear_btn.click(
            clear_chat,
            outputs=[chatbot_interface, msg]
        )
    
    return interface

# Créer l'interface
print("🎨 Création de l'interface Gradio...")
interface = create_gradio_interface()

# Lancer l'interface (décommentez pour utiliser)
# interface.launch(share=True, debug=True)

print("✅ Interface Gradio prête ! Décommentez la ligne ci-dessus pour la lancer.")

## 📊 **Métriques et Évaluation**

In [ ]:
def evaluate_chatbot_performance():
    """
    Évalue les performances du chatbot
    """
    print("📊 Évaluation des performances du chatbot...")
    
    # Questions de test avec réponses attendues
    test_cases = [
        {
            "question": "Que peux-tu me dire sur Hammamet ?",
            "expected_keywords": ["plage", "médina", "balnéaire", "tunisie"]
        },
        {
            "question": "Recommandations pour famille ?",
            "expected_keywords": ["famille", "piscine", "cuisine", "villa"]
        },
        {
            "question": "Meilleure période pour visiter ?",
            "expected_keywords": ["avril", "juin", "septembre", "novembre"]
        }
    ]
    
    results = []
    
    for test_case in test_cases:
        question = test_case["question"]
        expected_keywords = test_case["expected_keywords"]
        
        # Obtenir la réponse
        response = chatbot.chat(question)
        answer = response["answer"].lower()
        
        # Calculer le score de pertinence
        found_keywords = sum(1 for keyword in expected_keywords if keyword in answer)
        relevance_score = found_keywords / len(expected_keywords)
        
        results.append({
            "question": question,
            "relevance_score": relevance_score,
            "confidence": response["confidence"],
            "sources_used": len(response["sources"])
        })
        
        print(f"❓ {question}")
        print(f"   📈 Pertinence: {relevance_score:.2f}")
        print(f"   🎯 Confiance: {response['confidence']:.2f}")
        print(f"   📚 Sources: {len(response['sources'])}")
        print()
    
    # Statistiques globales
    avg_relevance = np.mean([r["relevance_score"] for r in results])
    avg_confidence = np.mean([r["confidence"] for r in results])
    
    print(f"📊 RÉSULTATS GLOBAUX:")
    print(f"   🎯 Pertinence moyenne: {avg_relevance:.2f}")
    print(f"   💪 Confiance moyenne: {avg_confidence:.2f}")
    
    return results

# Évaluer les performances
performance_results = evaluate_chatbot_performance()

## 📝 **Résumé et Prochaines Étapes**

In [ ]:
def print_summary():
    """
    Affiche un résumé du notebook et les prochaines étapes
    """
    print("""
🎉 CHATBOT LANGCHAIN + HUGGINGFACE CRÉÉ AVEC SUCCÈS !
=====================================================

✅ FONCTIONNALITÉS IMPLÉMENTÉES :
   🤖 Modèle HuggingFace intégré via LangChain
   🧠 Mémoire conversationnelle persistante
   🔍 RAG avec base de connaissances vectorielle
   📚 Embeddings multilingues pour la recherche sémantique
   🎨 Interface Gradio interactive
   📊 Système d'évaluation des performances

🛠️ TECHNOLOGIES UTILISÉES :
   • LangChain : Framework pour applications LLM
   • HuggingFace Transformers : Modèles de langage
   • FAISS : Base de données vectorielle
   • Sentence Transformers : Embeddings sémantiques
   • Gradio : Interface utilisateur interactive

🚀 PROCHAINES ÉTAPES RECOMMANDÉES :
   1. Tester avec différents modèles HuggingFace
   2. Enrichir la base de connaissances avec plus de données
   3. Implémenter des métriques d'évaluation avancées
   4. Déployer sur Hugging Face Spaces ou Streamlit Cloud
   5. Ajouter support pour d'autres langues
   6. Intégrer avec une API de données Airbnb réelles

🎯 VOTRE CHATBOT EST PRÊT POUR LA PRODUCTION !
    """)

# Afficher le résumé
print_summary()